In [1]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
# from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

regist=False

In [2]:

from detectron2.structures import BoxMode

datasets_path="../../datasets/segBuildings/"

def get_buildings_dicts(img_dir):
    json_file = os.path.join(img_dir, "annotation_train.json")
    with open(json_file) as f:
        imgs_anns = json.load(f)

    dataset_dicts = []
    for idx, v in enumerate(imgs_anns['features']):

        if (v['geometry']==None):
            continue

        record = {}

        filename = os.path.join(img_dir, v["img"])

        height, width = cv2.imread(filename).shape[:2]

        record["file_name"] = filename
        record["image_id"] = idx
        record["height"] = height
        record["width"] = width

        annos = v["geometry"]['coordinates']
        objs = []
    #     print(annos)
    #     break
        for anno in annos:
            (minX,maxY) = (min(anno,key=lambda tup:tup[0])[0],max(anno,key=lambda tup:tup[1])[1])
            (maxX,minY) = (max(anno,key=lambda tup:tup[0])[0],min(anno,key=lambda tup:tup[1])[1])
            poly = [p for x in anno for p in x]
            obj = {
                "bbox": [minX, minY, maxX, maxY],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": 0,
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts



In [3]:
if not regist:
    for d in ["train", "val"]:
        DatasetCatalog.register("segBuildings_" + d, lambda d=d: get_buildings_dicts(datasets_path + d))
        regist=True

dic =get_buildings_dicts(datasets_path + 'train')

segBuildings_metadata = MetadataCatalog.get("segBuildings__train")


In [4]:
## show img
dataset_dicts = get_buildings_dicts(datasets_path+"/train/")
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=segBuildings_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    cv2.imshow("trian im",out.get_image()[:, :, ::-1])
    cv2.waitKey(0)

cv2.waitKey()
cv2.destroyAllWindows()